In [1]:
from utz import *

In [2]:
src_bkt = 'tripdata'
dst_bkt = 'ctbk'
dst_root = None

from boto3 import client
from botocore import UNSIGNED
from botocore.client import Config
s3 = client('s3', config=Config())

In [4]:
jc5 = read_csv('s3://ctbk/csvs/JC-202105-citibike-tripdata.csv', dtype=str)
jc5

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,74A4206E7487CBC9,docked_bike,2021-05-23 16:51:00,2021-05-23 18:38:21,9 St HBLR - Jackson St & 8 St,HB305,9 St HBLR - Jackson St & 8 St,HB305,40.747907,-74.038411,40.74790720968899,-74.03841152787209,casual
1,58EEE2950FFE01CE,docked_bike,2021-05-31 16:54:47,2021-05-31 16:55:28,9 St HBLR - Jackson St & 8 St,HB305,9 St HBLR - Jackson St & 8 St,HB305,40.747907,-74.038411,40.74790720968899,-74.03841152787209,member
2,1429D912C16EEE59,docked_bike,2021-05-25 16:19:34,2021-05-25 17:03:06,9 St HBLR - Jackson St & 8 St,HB305,9 St HBLR - Jackson St & 8 St,HB305,40.747907,-74.038411,40.74790720968899,-74.03841152787209,casual
3,FE9C5B74167CBCCD,docked_bike,2021-05-22 17:32:19,2021-05-22 17:41:27,9 St HBLR - Jackson St & 8 St,HB305,Grand St & 2 St,HB405,40.747907,-74.038411,40.73913,-74.03618,casual
4,B88D37626F000BBA,docked_bike,2021-05-14 09:48:34,2021-05-14 10:17:36,Union St,JC051,Newark Ave,JC032,40.718211,-74.083639,40.72152515,-74.046304543,casual
...,...,...,...,...,...,...,...,...,...,...,...,...,...
45038,62FF3FEDAAD7FCB3,docked_bike,2021-05-31 12:46:22,2021-05-31 12:52:29,Clinton St & 7 St,HB303,7 St & Monroe St,HB304,40.74542,-74.03332,40.746413,-74.037977,casual
45039,E742AEA7068E0E4A,docked_bike,2021-05-31 16:34:19,2021-05-31 16:59:21,Clinton St & 7 St,HB303,Clinton St & 7 St,HB303,40.74542,-74.03332,40.74542,-74.03332,casual
45040,E769F9EE96D6433A,docked_bike,2021-05-17 22:52:35,2021-05-17 23:13:37,Astor Place,JC077,Astor Place,JC077,40.719282,-74.071261,40.71928220070702,-74.07126188278198,casual
45041,EC973614DCA179A2,docked_bike,2021-05-31 20:48:58,2021-05-31 21:16:02,Clinton St & 7 St,HB303,Clinton St & 7 St,HB303,40.74542,-74.03332,40.74542,-74.03332,casual


In [12]:
d5.columns

Index(['ride_id', 'rideable_type', 'started_at', 'ended_at',
       'start_station_name', 'start_station_id', 'end_station_name',
       'end_station_id', 'start_lat', 'start_lng', 'end_lat', 'end_lng',
       'member_casual'],
      dtype='object')

In [8]:
d5 = read_csv('s3://ctbk/csvs/202105-citibike-tripdata.csv', dtype=str)
d5

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,E12103A91226099A,docked_bike,2021-05-06 17:25:01,2021-05-06 17:45:25,Broadway & W 60 St,6948.11,1 Ave & E 78 St,7020.09,40.769155,-73.981918,40.77140426,-73.9535166,member
1,26A3DC47FE0EA3A3,docked_bike,2021-05-13 12:48:08,2021-05-13 13:07:37,Broadway & W 25 St,6173.08,E 2 St & Avenue B,5515.02,40.742868,-73.989186,40.72217444,-73.98368779,member
2,A99F2E1D627B088F,docked_bike,2021-05-16 08:30:13,2021-05-16 08:45:47,46 Ave & 5 St,6286.02,34th Ave & Vernon Blvd,6873.01,40.74731,-73.95451,40.76535380385866,-73.93986314535141,member
3,43E79A45997B7390,docked_bike,2021-05-01 08:38:14,2021-05-01 08:54:27,46 Ave & 5 St,6286.02,34th Ave & Vernon Blvd,6873.01,40.74731,-73.95451,40.76535380385866,-73.93986314535141,member
4,8B3CC649F4F58816,docked_bike,2021-05-09 08:12:31,2021-05-09 08:27:05,46 Ave & 5 St,6286.02,34th Ave & Vernon Blvd,6873.01,40.74731,-73.95451,40.76535380385866,-73.93986314535141,member
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2724160,766BB4E17F05DBB4,docked_bike,2021-05-26 20:48:14,2021-05-26 21:02:01,W 25 St & 6 Ave,6215.04,10 Ave & W 14 St,6157.04,40.743954,-73.991448,40.741981599940225,-74.00831580162048,member
2724161,3A74372C4A83AF31,docked_bike,2021-05-22 16:28:54,2021-05-22 16:43:55,W 25 St & 6 Ave,6215.04,10 Ave & W 14 St,6157.04,40.743954,-73.991448,40.741981599940225,-74.00831580162048,member
2724162,8FF3997F7C47E8FA,docked_bike,2021-05-12 13:46:29,2021-05-12 13:54:12,E 4 St & 2 Ave,5593.04,Canal St & Rutgers St,5303.08,40.72628,-73.98978,40.71427487,-73.98990025,member
2724163,FFB1C29E91CC7253,docked_bike,2021-05-08 00:15:34,2021-05-08 00:25:19,Lewis Ave & Decatur St,4237.01,Central Ave & Woodbine St,4632.10,40.681459,-73.934903,40.69296,-73.91605,member


In [13]:
d5.start_station_id.astype(float)

ValueError: could not convert string to float: 'Lab - NYC'

In [10]:
d5.rideable_type.value_counts()

docked_bike      2724017
electric_bike        148
Name: rideable_type, dtype: int64

In [11]:
d2.rideable_type.value_counts()

docked_bike    649983
Name: rideable_type, dtype: int64

In [9]:
d5.member_casual.value_counts()

member    1914217
casual     809948
Name: member_casual, dtype: int64

In [5]:
d2 = read_csv('s3://ctbk/csvs/202102-citibike-tripdata.csv', dtype=str)
d2

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,BBA33D73DECE976F,docked_bike,2021-02-26 16:38:54,2021-02-26 16:44:37,E 84 St & Park Ave,7243.04,E 78 St & 2 Ave,7057.07,40.778626,-73.95772,40.7727966,-73.95577801,casual
1,B63D7AFF9AC5B6D4,docked_bike,2021-02-17 11:09:11,2021-02-17 11:26:47,Macon St & Nostrand Ave,4214.03,Bond St & Fulton St,4479.06,40.680983,-73.950047,40.68962188790333,-73.98304268717766,member
2,52B829195C469D99,docked_bike,2021-02-26 18:33:29,2021-02-26 19:05:41,Macon St & Nostrand Ave,4214.03,Lefferts Pl & Franklin Ave,4222.02,40.680983,-73.950047,40.680342423,-73.9557689392,casual
3,19C84ECA2B468476,docked_bike,2021-02-26 12:48:35,2021-02-26 13:07:24,Macon St & Nostrand Ave,4214.03,Bond St & Fulton St,4479.06,40.680983,-73.950047,40.68962188790333,-73.98304268717766,member
4,C0DDB771E70D9DF5,docked_bike,2021-02-25 17:23:22,2021-02-25 17:28:20,Madison Ave & E 26 St,6131.12,W 37 St & 5 Ave,6398.06,40.742685,-73.986713,40.75038,-73.98339,member
...,...,...,...,...,...,...,...,...,...,...,...,...,...
649978,84DA58E73B2DF079,docked_bike,2021-02-12 16:01:17,2021-02-12 16:13:53,Irving Ave & DeKalb Ave,4898.02,Irving Ave & DeKalb Ave,4898.02,40.7027,-73.92095,40.7027,-73.92095,member
649979,3A14299B4F7FA1CF,docked_bike,2021-02-17 18:03:24,2021-02-17 18:06:39,E 47 St & Park Ave,6584.12,E 47 St & Park Ave,6584.12,40.755102,-73.974986,40.75510267,-73.97498696,member
649980,7B86EE3DC7E026BC,docked_bike,2021-02-28 18:58:31,2021-02-28 19:12:51,Cedar St & Evergreen Ave,4721.01,Cedar St & Evergreen Ave,4721.01,40.69671,-73.92807,40.69671,-73.92807,casual
649981,D665B8623FC01285,docked_bike,2021-02-25 09:00:41,2021-02-25 09:13:56,Bedford Ave & Bergen St,4066.15,Cedar St & Evergreen Ave,4721.01,40.676368,-73.952918,40.69671,-73.92807,casual


In [6]:
d1 = read_csv('s3://ctbk/csvs/202101-citibike-tripdata.csv', dtype=str)
d1

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender
0,2513,2021-01-01 00:00:11.9020,2021-01-01 00:42:05.2260,3581,Underhill Ave & Lincoln Pl,40.6740123,-73.9671457,3581,Underhill Ave & Lincoln Pl,40.6740123,-73.9671457,47812,Customer,1969,0
1,2519,2021-01-01 00:00:15.0960,2021-01-01 00:42:14.9780,3581,Underhill Ave & Lincoln Pl,40.6740123,-73.9671457,3581,Underhill Ave & Lincoln Pl,40.6740123,-73.9671457,47571,Customer,1969,0
2,1207,2021-01-01 00:00:28.9300,2021-01-01 00:20:36.6510,3144,E 81 St & Park Ave,40.77677702,-73.9590097,3724,7 Ave & Central Park South,40.7667405590595,-73.9790689945221,37451,Subscriber,2002,1
3,2506,2021-01-01 00:00:32.7130,2021-01-01 00:42:19.3980,3581,Underhill Ave & Lincoln Pl,40.6740123,-73.9671457,3581,Underhill Ave & Lincoln Pl,40.6740123,-73.9671457,48884,Customer,2002,1
4,959,2021-01-01 00:00:35.3650,2021-01-01 00:16:34.6010,534,Water - Whitehall Plaza,40.70255065,-74.0127234,332,Cherry St,40.71219906,-73.97948148,26837,Customer,2002,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1095341,569,2021-01-31 23:54:00.3610,2021-02-01 00:03:29.8190,3300,Prospect Park West & 8 St,40.66514681533792,-73.97637605667114,3395,Henry St & W 9 St,40.6763744,-74.00324957,47613,Subscriber,1958,1
1095342,1439,2021-01-31 23:55:50.4170,2021-02-01 00:19:49.8980,3081,Graham Ave & Grand St,40.711863,-73.944024,307,Canal St & Rutgers St,40.71427487,-73.98990025,36798,Subscriber,1988,1
1095343,2619,2021-01-31 23:56:14.9540,2021-02-01 00:39:54.3500,447,8 Ave & W 52 St,40.76370739,-73.9851615,4058,Madison Av & E 51 St,40.75863,-73.97513,48698,Subscriber,1985,1
1095344,83,2021-01-31 23:59:19.8260,2021-02-01 00:00:43.5840,3462,E 44 St & 2 Ave,40.75118387463277,-73.97138714790344,3462,E 44 St & 2 Ave,40.75118387463277,-73.97138714790344,49619,Subscriber,1969,1


In [3]:
resp = s3.list_objects_v2(Bucket=src_bkt)
contents = pd.DataFrame(resp['Contents'])
zips = contents[contents.Key.str.endswith('.zip')]
zips

,Key,LastModified,ETag,Size,StorageClass
0,201306-citibike-tripdata.zip,2018-04-30 13:18:55+00:00,"""b520a12de58eea58a3586f89bfcfbd9d-2""",16785103,STANDARD
1,201307-201402-citibike-tripdata.zip,2017-01-18 22:23:25+00:00,"""7b3b260b2ab2e5349320121d04bd821c-22""",178262576,STANDARD
2,201307-citibike-tripdata.zip,2017-01-18 22:23:27+00:00,"""dd3e6fd5f91715b31eae72868086c08c-4""",27074629,STANDARD
3,201308-citibike-tripdata.zip,2017-01-18 22:23:27+00:00,"""2f661063576734f614b9f1d6bba0ec59-4""",32090869,STANDARD
4,201309-citibike-tripdata.zip,2017-01-18 22:23:27+00:00,"""a42f947db7bd14e423a7dbfbb11596a1-4""",33155593,STANDARD
...,...,...,...,...,...
161,JC-202101-citibike-tripdata.csv.zip,2021-02-08 15:11:26+00:00,"""02e5da50db92added528f438587bb1e7""",313806,STANDARD
162,JC-202102-citibike-tripdata.csv.zip,2021-06-11 17:23:13+00:00,"""53faf8f5323d6ea79f79fdcce5f4894d""",145668,STANDARD
163,JC-202103-citibike-tripdata.csv.zip,2021-06-11 17:23:13+00:00,"""39de5df5ab7f113d39244aa3722806d5""",473774,STANDARD
164,JC-202104-citibike-tripdata.csv.zip,2021-06-11 17:23:14+00:00,"""400e559b834a3c02d6a6c57bc4dd9d51""",650251,STANDARD


In [2]:
src_bkt = 'tripdata'
dst_bkt = 'ctbk'
dst_root = None

env['AWS_PROFILE'] = 
from boto3 import client
from botocore import UNSIGNED
from botocore.client import Config
s3 = client('s3', config=Config())

In [4]:
rgx = r'^(?P<JC>JC-)?(?P<year>\d{4})(?P<month>\d{2})[ \-]citibike-tripdata?(?P<csv>\.csv)?(?P<zip>\.zip)?$'

In [5]:
fields = {
  'Trip Duration',
  'Start Time',
  'Stop Time',
  'Start Station ID',
  'Start Station Name',
  'Start Station Latitude',
  'Start Station Longitude',
  'End Station ID',
  'End Station Name',
  'End Station Latitude',
  'End Station Longitude',
  'Bike ID',
  'User Type',
  'Birth Year',
  'Gender'
}
def normalize_field(f): return sub(r'\s', '', f.lower())
normalize_fields_map = { normalize_field(f): f for f in fields }
normalize_fields_map

{'stoptime': 'Stop Time',
 'tripduration': 'Trip Duration',
 'endstationlongitude': 'End Station Longitude',
 'birthyear': 'Birth Year',
 'startstationlongitude': 'Start Station Longitude',
 'endstationname': 'End Station Name',
 'usertype': 'User Type',
 'gender': 'Gender',
 'startstationname': 'Start Station Name',
 'endstationlatitude': 'End Station Latitude',
 'startstationid': 'Start Station ID',
 'endstationid': 'End Station ID',
 'starttime': 'Start Time',
 'bikeid': 'Bike ID',
 'startstationlatitude': 'Start Station Latitude'}

In [6]:
def normalize_fields(df):
    return df.rename(columns={
        col: normalize_fields_map[normalize_field(col)]
        for col in df.columns
    })

In [7]:
from zipfile import ZipFile

In [8]:
from botocore.client import ClientError
def s3_exists(Bucket, Key, s3=None):
    if not s3:
        s3 = client('s3', config=Config(signature_version=UNSIGNED))
    try:
        s3.head_object(Bucket=Bucket, Key=Key)
        return True
    except ClientError:
        return False

In [9]:
def to_parquet(zip_key, error='warn', overwrite=False):
    name = basename(zip_key)
    m = match(rgx, name)
    if not m:
        msg = f'Unrecognized key: {name}'
        if error == 'warn':
            print(msg)
            return msg
        else:
            raise Exception(msg)
    base, ext = splitext(zip_key)
    assert ext == '.zip'
    if base.endswith('.csv'):
        base = splitext(base)[0]

    # normalize the dst path; a few src files have typos/inconsistencies
    base = '%s%s%s-citibike-tripdata' % (m['JC'] or '', m['year'], m['month'])
    if dst_root is None:
        dst_key = f'{base}.parquet'
    else:
        dst_key = f'{dst_root}/{base}.parquet'
    dst = f's3://{dst_bkt}/{dst_key}'
    s3 = client('s3', config=Config())
    if s3_exists(dst_bkt, dst_key, s3=s3):
        if overwrite:
            msg = f'Overwrote {dst}'
            print(f'Overwriting {dst}')
        else:
            msg = f'Found {dst}; skipping'
            print(msg)
            return msg
    else:
        msg = f'Wrote {dst}'

    with TemporaryDirectory() as d:
        zip_path = f'{d}/{base}.zip'
        pqt_path = f'{d}/{base}.parquet'
        s3.download_file(src_bkt, zip_key, zip_path)
        z = ZipFile(zip_path)
        names = z.namelist()
        print(f'{name}: zip names: {names}')
        [ name ] = [ f for f in names if f.endswith('.csv') and not f.startswith('_') ]
        with z.open(name,'r') as i:
            df = pd.read_csv(i)
            df = normalize_fields(df)
            df = df.astype({'Start Time':'datetime64[ns]','Stop Time':'datetime64[ns]'})
            df.to_parquet(pqt_path)

        s3.upload_file(pqt_path, dst_bkt, dst_key)

        return msg

In [10]:
from joblib import delayed, Parallel
parallel = Parallel(n_jobs=cpu_count())

In [11]:
print('\n'.join(parallel(delayed(to_parquet)(f) for f in zips.Key.values)))

Found s3://ctbk/201306-citibike-tripdata.parquet; skipping
Unrecognized key: 201307-201402-citibike-tripdata.zip
Found s3://ctbk/201307-citibike-tripdata.parquet; skipping
Found s3://ctbk/201308-citibike-tripdata.parquet; skipping
Found s3://ctbk/201309-citibike-tripdata.parquet; skipping
Found s3://ctbk/201310-citibike-tripdata.parquet; skipping
Found s3://ctbk/201311-citibike-tripdata.parquet; skipping
Found s3://ctbk/201312-citibike-tripdata.parquet; skipping
Found s3://ctbk/201401-citibike-tripdata.parquet; skipping
Found s3://ctbk/201402-citibike-tripdata.parquet; skipping
Found s3://ctbk/201403-citibike-tripdata.parquet; skipping
Found s3://ctbk/201404-citibike-tripdata.parquet; skipping
Found s3://ctbk/201405-citibike-tripdata.parquet; skipping
Found s3://ctbk/201406-citibike-tripdata.parquet; skipping
Found s3://ctbk/201407-citibike-tripdata.parquet; skipping
Found s3://ctbk/201408-citibike-tripdata.parquet; skipping
Found s3://ctbk/201409-citibike-tripdata.parquet; skipping
Fo